### Import packages and initialize variables (example here is done for Well11)

In [1]:
from sklearn.neighbors import NearestNeighbors as KNN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt

In [2]:
sample = 'well11'
is_sagittal = False
racrna_file = 'data/Well11/racRNA_spot_meta_all.csv'
spot_file = 'data/Well11/well11_spot_meta.csv'
racrna_output_file = "output/classified_racrna/Well11_classified_racrna.csv"
cell_output_file = 'output/cell_info/Well11_info_by_cell.csv'

### Read in data for racRNA spot calling

In [3]:
racrna_data = pd.read_csv(racrna_file)

In [4]:
spot_data = pd.read_csv(spot_file)

### Auxiliary data for plotting racRNA

In [5]:
sns.set(rc={'axes.facecolor':'black', 'figure.facecolor':'black'})
sns.set_style({'axes.grid': False})

In [6]:
def plot_base(x1, y1, x2, y2, s=20, cell_rac=True, palette_type='mako'):
    racrna_x_cond = (racrna_data['spot_merged_1'] >= x1) & (racrna_data['spot_merged_1'] < x2)
    racrna_y_cond = (racrna_data['spot_merged_2'] >= y1) & (racrna_data['spot_merged_2'] < y2)
    spot_x_cond = (spot_data['spot_merged_1'] >= x1) & (spot_data['spot_merged_1'] < x2)
    spot_y_cond = (spot_data['spot_merged_2'] >= y1) & (spot_data['spot_merged_2'] < y2)
    racrna_subset = racrna_data[racrna_x_cond & racrna_y_cond].reset_index()
    spot_subset = spot_data[spot_x_cond & spot_y_cond].reset_index()
    plt.figure(figsize=(20,20))
    if not cell_rac:
        palette = sns.color_palette(
            palette_type, 
            len(spot_subset['cellid_idx'].unique())
        )
        sns.scatterplot(
            x='spot_merged_1', 
            y='spot_merged_2', 
            data=spot_subset, 
            s=s,
            hue='cellid_idx', 
            palette=palette, 
            legend=False
        )
        sns.scatterplot(
            x='spot_merged_1', 
            y='spot_merged_2', 
            data=racrna_subset, 
            s=s,
            marker='x',
            color='r', 
            legend=False
        )
    else:
        data = pd.DataFrame({
            'spot_merged_1': list(spot_subset['spot_merged_1']) + list(racrna_subset['spot_merged_1']),
            'spot_merged_2': list(spot_subset['spot_merged_2']) + list(racrna_subset['spot_merged_2']),
            'cellid_idx': list(spot_subset['cellid_idx']) + list(racrna_subset['cellid_idx']),
            'is_racrna': [0] * spot_subset.shape[0] + [1] * racrna_subset.shape[0]
        })
        palette = sns.color_palette(
            palette_type, 
            len(data['cellid_idx'].unique())
        )
        plt.figure(figsize=(20, 20))
        sns.scatterplot(
            x='spot_merged_1', 
            y='spot_merged_2', 
            data=data, 
            s=s,
            hue='cellid_idx', 
            style='is_racrna',
            palette=palette, 
            legend=False
        )
    sns.despine()
    plt.show()

In [7]:
def plot_racrna(x1, y1, x2, y2, s=20, palette_type='mako'):
    racrna_x_cond = (racrna_data['spot_merged_1'] >= x1) & (racrna_data['spot_merged_1'] < x2)
    racrna_y_cond = (racrna_data['spot_merged_2'] >= y1) & (racrna_data['spot_merged_2'] < y2)
    racrna_subset = racrna_data[racrna_x_cond & racrna_y_cond].reset_index()
    plt.figure(figsize=(20,20))
    palette = sns.color_palette(
        palette_type, 
        len(racrna_subset['cellid_idx'].unique())
    )
    plt.figure(figsize=(20, 20))
    sns.scatterplot(
        x='spot_merged_1', 
        y='spot_merged_2', 
        data=racrna_subset, 
        s=s,
        hue='cellid_idx',
        palette=palette, 
        legend=False
    )
    sns.despine()
    plt.show()

### KNN classification code

In [8]:
CUTOFF = 10
VERY_CLOSE = 3.5

In [9]:
X = spot_data[['spot_merged_1', 'spot_merged_2', 'spot_merged_3']].to_numpy()
Y = spot_data['cellid_idx']
Xpred = racrna_data[['spot_merged_1', 'spot_merged_2', 'spot_merged_3']].to_numpy()
for i in range(X.shape[0]):
    X[i,2] /= 10
for i in range(Xpred.shape[0]):
    Xpred[i,2] /= 10

In [10]:
knn = KNN(n_neighbors=3).fit(X)
distances, indices = knn.kneighbors(Xpred)

In [11]:
n_pred = Xpred.shape[0]

In [12]:
cell_type = []
for i in range(n_pred):
    if distances[i,0] < VERY_CLOSE or (distances[i,2] < CUTOFF and Y[indices[i,0]] == Y[indices[i,1]] and Y[indices[i,1]] == Y[indices[i,2]]):
        cell_type.append(Y[indices[i,0]])
    else:
        cell_type.append(-1)

In [13]:
print(f"Of {len(cell_type)} racRNA spots, {len(cell_type) - int(np.sum(np.array(cell_type) == -1))} were placed into a cell.")

Of 12504365 racRNA spots, 5822040 were placed into a cell.


In [14]:
racrna_data.loc[:,'cellid_idx'] = cell_type

### Group data by cell

In [15]:
ctype_file = 'data/pd_tissue.csv'

In [16]:
ctype_data = pd.read_csv(ctype_file, low_memory=False)[['orginindex', 'sample']]
ctype_data = ctype_data.loc[ctype_data['sample'] == sample].reset_index()[['orginindex']]
ctype_data['orginindex'] = ctype_data['orginindex'].astype(int)

In [17]:
verbosity = False

n_cells = np.max(ctype_data['orginindex']) + 1
if is_sagittal:
    racrna_counter = np.zeros((n_cells, 4))
else:
    racrna_counter = np.zeros(n_cells)
cellids = racrna_data['cellid_idx']
geneids = racrna_data['geneid']
for i in range(racrna_data.shape[0]):
    cellid_idx = cellids[i]
    geneid = geneids[i]
    if cellid_idx != -1 and cellid_idx < n_cells:
        if is_sagittal:
            racrna_counter[cellid_idx][geneid-1] += 1
        else:
            racrna_counter[cellid_idx] += 1
    elif cellid_idx >= n_cells and verbosity:
        print("Found unknown cell: ", cellid_idx, f"(gene ID {geneid})") # some (usually <= 5) cells were not in cell type data

In [18]:
to_df = {
    'cellid_idx': [],
    'racRNA uptake': []
}

if is_sagittal:
    to_df['geneid'] = []

for idx in range(n_cells):
    if np.sum(racrna_counter[idx]) > 0:
        if is_sagittal:
            for i in range(4):
                to_df['cellid_idx'].append(idx)
                to_df['racRNA uptake'].append(racrna_counter[idx,i])
                to_df['geneid'].append(i+1)
        else:
            to_df['cellid_idx'].append(idx)
            to_df['racRNA uptake'].append(racrna_counter[idx])

In [19]:
cellinfo_df = pd.DataFrame(to_df)

### Output data

In [20]:
racrna_data.to_csv(racrna_output_file)

In [21]:
cellinfo_df.to_csv(cell_output_file)